In [1]:
#CoraFull-SAGE-1

import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dgl-inbuilt-graphs/Pubmed.bin
/kaggle/input/dgl-inbuilt-graphs/CoraFull.bin
/kaggle/input/dgl-inbuilt-graphs/CoauthorPhysics.bin
/kaggle/input/dgl-inbuilt-graphs/Citeseer.bin
/kaggle/input/dgl-inbuilt-graphs/CoauthorCS.bin


In [2]:
!nvcc --version
import torch

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [3]:
!pip install dgl-cu102
import dgl

     |████████████████████████████████| 36.8 MB 14.4 MB/s 
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


In [4]:
#loader
from dgl import save_graphs, load_graphs

def load_graph(name, split):
    k_path = r"/kaggle/input/dgl-inbuilt-graphs/"
    graph = load_graphs(k_path + name)[0][0]
    label = graph.ndata['label']
    feat = graph.ndata['feat']
    in_feat = graph.ndata['feat'].shape[1]
    out_feat = int(max(label)+1)
    mask = torch.BoolTensor(graph.num_nodes())
    mask[:] = False
    split_idx = int((graph.num_nodes()*split))
    mask[:split_idx] = True
    train_mask = mask
    test_mask = torch.logical_not(train_mask)
    return graph, feat, label, train_mask, test_mask, in_feat, out_feat

In [5]:
g, features, labels, train_mask, test_mask, in_feat, out_feat = load_graph("CoraFull.bin",0.8)

In [6]:
import dgl
import torch
import torch.nn.functional as F
import time
import numpy as np
import psutil
dgl.seed(0)
torch.manual_seed(0)
dgl.random.seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

IN_FEAT = in_feat
H_FEAT = 128
OUT_FEAT = out_feat

BACKBONE = 'gcn' #for logging only

In [7]:
# set config

CONFIG = 1

if CONFIG == 0: #no ed
    EDROP_DA = False #dont care
    EDROPOUT = 0
elif CONFIG == 1: #random ed
    EDROP_DA = False
    EDROPOUT = 0.2
elif CONFIG == 2: #DAw ed
    EDROP_DA = True
    EDROPOUT = 0.2
else: #default config
    EDROP_DA = True
    EDROPOUT = 0.2
    
RUNS = 20
NDROPOUT = 0.5
EPOCHS = 200
NORM = False
N_BLOCKS = [2,4,6,8]
ROTATE_BY = 50
PRINT_EVERY = 10

In [8]:
import torch.nn as nn
import random
from random import randrange

class RandomEdgeDropout(nn.Module):
    def __init__(self, da=False, dropout = 0.2, device='cuda:0'):
        super(RandomEdgeDropout, self).__init__()
        self.dropout = dropout
        self.da = da

    def forward(self, g):
        g = g.local_var()
        g = g.remove_self_loop()
        
        if not self.da:
            num_edges2drop = int(g.num_edges()*self.dropout)
            edges2drop = [randrange(g.num_edges()) for _ in range(num_edges2drop)]
        else:
            droppable = [idx for idx,i in enumerate(g.edata['droppable'].tolist()) if i==1]
            num_edges2drop = int(len(droppable)*self.dropout)
            edges2drop = random.sample(droppable, num_edges2drop)
        
        g.remove_edges(torch.tensor(edges2drop).to(device))
        g = dgl.add_self_loop(g)
        return g

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv, SAGEConv

class Net(nn.Module):
    def __init__(
        self, 
        graph, 
        in_feat, 
        h_feat, 
        out_feat, 
        n_blocks,
        edropout = 0.2,
        edropout_degAw = True, 
        ndropout = 0.2,
        norm = True, 
        device = 'cuda:0'
        ):
        super(Net, self).__init__()
        self.in_feat = in_feat
        self.h_feat = h_feat
        self.out_feat = out_feat
        
        self.edropout = edropout
        self.edropout_degAw = edropout_degAw
        self.ndropout = ndropout
        
        self.norm = norm
        self.n_blocks = n_blocks
        self.g = graph
        
        self.device = device
        self.layers = nn.ModuleList()
        
        self.ed = RandomEdgeDropout(da=True, dropout = self.edropout, device=self.device)
        self.lrelu = nn.LeakyReLU(negative_slope=0.1, inplace=False)

        for idx in range(self.n_blocks-1):
            if (idx==0):
                # self.layers.append(GraphConv(self.in_feat, self.h_feat, norm='both', weight=True, bias=True))
                self.layers.append(SAGEConv(self.in_feat, self.h_feat,aggregator_type='gcn'))
                #SAGEConv
            else:
                # self.layers.append(GraphConv(self.h_feat, self.h_feat, norm='both', weight=True, bias=True))
                self.layers.append(SAGEConv(self.h_feat, self.h_feat,aggregator_type='gcn'))
        # self.layers.append(GraphConv(self.h_feat, self.out_feat, norm='both', weight=True, bias=True))
        self.layers.append(SAGEConv(self.h_feat, self.out_feat,aggregator_type='gcn'))
        self.fc = nn.Linear(self.out_feat, self.out_feat)
        self.bn = torch.nn.BatchNorm1d(self.out_feat)
    
    def forward(self, g, features):
        x = features
        for lyr in self.layers:
            with g.local_scope():
                if self.edropout != 0:
                    g = self.ed(g)
                x = self.lrelu(lyr(g, x))
                if self.ndropout != None:
                    x = F.dropout(x, self.ndropout, training=self.training)
        x = self.fc(x)
        if self.norm:
            x = self.bn(x)
        return x

In [10]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [11]:
g = dgl.add_self_loop(g)
g = g.to(device)
features = features.to(device)
labels = labels.to(device)

In [12]:
print(BACKBONE)
for N_BLOCK in N_BLOCKS:
    print('=' ,N_BLOCK,'layers ','='*42)
    best_scores =[]
    for run in range(RUNS):
        RUN_ID = run+1
        print('-' ,RUN_ID, '-'*50)

        net = Net(g, IN_FEAT, H_FEAT, OUT_FEAT, N_BLOCK, EDROPOUT, EDROP_DA, NDROPOUT, NORM, device).to(device)

        optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
        dur = []

        best_score = 0
        best_epoch = 0

        for epoch in range(EPOCHS):
            if epoch >=3:
                t0 = time.time()

            net.train()

            logits = net(g, features)

            logp = F.log_softmax(logits, 1)
            loss = F.nll_loss(logp[train_mask], labels[train_mask])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch >=3:
                dur.append(time.time() - t0)

            acc = evaluate(net, g, features, labels, test_mask)
            if epoch%PRINT_EVERY==0:
                print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(epoch, loss.item(), acc, np.mean(dur)))

            if best_score < acc:
                best_score = acc
                best_epoch = epoch

        print("Best Test Acc {:.4f} at Epoch {:05d}".format(best_score, best_epoch))
        best_scores.append(best_score)

        del net
        train_mask = torch.tensor(train_mask.tolist()[ROTATE_BY:] + train_mask.tolist()[:ROTATE_BY]).bool().to(device)
        test_mask = torch.tensor(test_mask.tolist()[ROTATE_BY:] + test_mask.tolist()[:ROTATE_BY]).bool().to(device)
        
    #summarize
    import statistics as stats

    best_scores.sort()
    best_scores.reverse()
    best_scores = best_scores[5:]
    best_scores = [score for score in best_scores]

    mean = stats.mean(best_scores)
    stdev = stats.stdev(best_scores)
    
    print("Summary:")
    print(round(mean,4),"±",round(stdev,4))

gcn
= 2 layers  ==========================================
- 1 --------------------------------------------------


/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 4.2428 | Test Acc 0.0472 | Time(s) nan
Epoch 00010 | Loss 3.0064 | Test Acc 0.3630 | Time(s) 0.0638
Epoch 00020 | Loss 1.7727 | Test Acc 0.5822 | Time(s) 0.0725
Epoch 00030 | Loss 1.2374 | Test Acc 0.6388 | Time(s) 0.0717
Epoch 00040 | Loss 0.9744 | Test Acc 0.6651 | Time(s) 0.0697
Epoch 00050 | Loss 0.8035 | Test Acc 0.6661 | Time(s) 0.0693
Epoch 00060 | Loss 0.6974 | Test Acc 0.6661 | Time(s) 0.0685
Epoch 00070 | Loss 0.6103 | Test Acc 0.6598 | Time(s) 0.0678
Epoch 00080 | Loss 0.5497 | Test Acc 0.6560 | Time(s) 0.0696
Epoch 00090 | Loss 0.5014 | Test Acc 0.6529 | Time(s) 0.0690
Epoch 00100 | Loss 0.4474 | Test Acc 0.6494 | Time(s) 0.0690
Epoch 00110 | Loss 0.4132 | Test Acc 0.6433 | Time(s) 0.0683
Epoch 00120 | Loss 0.3839 | Test Acc 0.6370 | Time(s) 0.0678
Epoch 00130 | Loss 0.3580 | Test Acc 0.6408 | Time(s) 0.0673
Epoch 00140 | Loss 0.3358 | Test Acc 0.6385 | Time(s) 0.0670
Epoch 00150 | Loss 0.3179 | Test Acc 0.6365 | Time(s) 0.0667
Epoch 00160 | Loss 0.2996 |